In [1]:
!pip install -q -U google-generativeai
import pathlib
import textwrap
from google.colab import userdata
import google.generativeai as genai
import pandas as pd

from IPython.display import display
from IPython.display import Markdown


def to_markdown(text):
  text = text.replace('•', '  *')
  return Markdown(textwrap.indent(text, '> ', predicate=lambda _: True))

# Used to securely store your API key
df = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/T20_WorldCup/T20_cleaned_comments.csv")
df['all_eng']
temp_list = []
count = 0
seq = 0
df_length = len(df['all_eng'])
if df_length < 300:
    seq = seq + 1
    for row in df['all_eng']:
      temp_list.append(row)

    dict = {'temp_col':temp_list,}
    df = pd.DataFrame(dict)
    df.to_csv('/content/drive/MyDrive/Colab Notebooks/gemini_AI/files/'+str(seq)+'.csv')

if df_length >= 300:
  for row in df['all_eng']:
    temp_list.append(row)
    count = count + 1

    if count == 300:
      seq = seq + 1
      dict = {'temp_col':temp_list,}
      df = pd.DataFrame(dict)
      df.to_csv('/content/drive/MyDrive/Colab Notebooks/gemini_AI/files/'+str(seq)+'.csv')
      count = 0
      temp_list = []

# Or use `os.getenv('GOOGLE_API_KEY')` to fetch an environment variable.
GOOGLE_API_KEY=userdata.get('GOOGLE_API_KEY')

genai.configure(api_key=GOOGLE_API_KEY)

model = genai.GenerativeModel('gemini-1.5-pro-latest')

#%%time
#text = ''
def textSenti(text1):
  response = model.generate_content("Answer the Sentiment of the text only as Positive, Negative or Neutral: "+str(text1))
  return(response.text)

import time
for i in range(38, seq + 1):
  negList = []
  sentList = []
  flag = 0
  df_1 = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/gemini_AI/files/'+str(i)+'.csv')
  for text1 in df_1['temp_col']:
    flag = flag + 1
    time.sleep(1) # Sleep for 1 seconds
    try:
      time.sleep(1)
      senti = textSenti(text1)
      time.sleep(1)
      print(text1, senti, flag)
      if senti == 'Negative' or 'Positive' or 'Neutral':
        negList.append(text1)
        sentList.append(senti)
    except:
      print("Error")
    time.sleep(1) # Sleep for 1 seconds

  dict = {'Negative_Sentence' : negList,
          'Sentiment': sentList,}
  df = pd.DataFrame(dict)
  df.to_csv('/content/drive/MyDrive/Colab Notebooks/gemini_AI/labelled_text/'+str(i)+'.csv')
  print(i)



Streaming output truncated to the last 5000 lines.
 204
jihadi har gye bosdiwale Negative
 205
win or loss is a part of game Neutral
 206
memorable self destruction by pakistan Negative
 207
ye naseem shah har baar harne pe ro deta hai Negative
 208
islam ki har hui jai shree ram Negative
 209
poojeets won Positive
 210
thank you icc for uploading this Positive
 211
india is the best team Positive
 212
this quality video and level of editing we expect from icc well done Positive
 213
icc montage videos are best videos Positive
 214
icc never missed a chance to open the clothes of pak Negative
 215
re neet Neutral
 216
shittest country on earth pakistanpathetic team lol Negative
 217
boring Negative
 218
gote muh main ageya tha Negative
 219
upload highlights Neutral
 220
magical match Positive
 221
pak team successfully qualified for lahore bus stand Negative
 222
bzimbabar b Neutral
 223
we need a biopic on bumrah asap enough with batters biopic Positive
 224
hindustan zindabad jai sh

In [2]:
import pandas as pd
import os

# Define paths
base_path = "/content/drive/MyDrive/Colab Notebooks/gemini_AI"
input_folder = os.path.join(base_path, "labelled_text")
output_folder = os.path.join(base_path, "senti_count")
output_file = os.path.join(output_folder, "merged_labels.csv")

# Create output folder if it doesn't exist
os.makedirs(output_folder, exist_ok=True)

# Read and merge all CSV files
df_list = []
for i in range(1, 47):
    file_path = os.path.join(input_folder, f"{i}.csv")
    df_list.append(pd.read_csv(file_path))

df_merged = pd.concat(df_list, ignore_index=True)

# Save the merged DataFrame to the new folder
df_merged.to_csv(output_file, index=False)

print(f"Merged file saved at: {output_file}")

# Sentiment count and percentage calculation
pcount = (df_merged['Sentiment'] == 'Positive\n').sum()
ncount = (df_merged['Sentiment'] == 'Negative\n').sum()
neucount = (df_merged['Sentiment'] == 'Neutral\n').sum()
total = pcount + ncount + neucount

ppercent = (pcount / total) * 100 if total > 0 else 0
npercent = (ncount / total) * 100 if total > 0 else 0
neupercent = (neucount / total) * 100 if total > 0 else 0

print(f'Positive: {pcount} ({ppercent:.2f}%)')
print(f'Negative: {ncount} ({npercent:.2f}%)')
print(f'Neutral: {neucount} ({neupercent:.2f}%)')


Merged file saved at: /content/drive/MyDrive/Colab Notebooks/gemini_AI/senti_count/merged_labels.csv
Positive: 7635 (57.20%)
Negative: 2895 (21.69%)
Neutral: 2819 (21.12%)
